In [ ]:
from g4f.client import Client

In [ ]:
from sklearn.neighbors import BallTree
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
class BERTEmbedder:
    def __init__(self, model_name="sentence-transformers/stsb-bert-base", device=None):
        self.device = device or 'cpu'
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)
        self.model.eval()
        
        self.do_lower_case = getattr(self.tokenizer, 'do_lower_case', False)

    def text_to_embedding(self, texts, pooling='mean', normalize=False):
        is_single = isinstance(texts, str)
        texts = [texts] if is_single else texts
        
        inputs = self.tokenizer(
            texts,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=128
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            
        if pooling == 'mean':
            mask = inputs['attention_mask'].unsqueeze(-1)
            embeddings = (outputs.last_hidden_state * mask).sum(1) / mask.sum(1).clamp(min=1e-9)
        elif pooling == 'cls':
            embeddings = outputs.last_hidden_state[:, 0, :]
        else:
            raise ValueError("Invalid pooling method")
            
        if normalize:
            embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
            
        return embeddings.cpu().numpy()[0] if is_single else embeddings.cpu().numpy()


In [ ]:
client = Client()
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Hello"}],
    web_search=False
)
print(response.choices[0].message.content)

Hey there! What’s on your mind today?


In [ ]:
def retrive_docs(query : str, k : int = 5):
    query_embeding = BERTEmbedder.encode(query)